In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np

In [5]:
with open('Korean_movie_reviews_2016_small.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    texts, labels = zip(*docs) # 둘을 분리해서 별도의 list 변수로 저장  
# 영화평 데이터는 text, 종속변수는 lables

In [7]:
print(sum(labels))
print(len(labels))

11994
22909


In [9]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.1, random_state=0)

In [11]:
# CounterVectorizer 클래스를 이용한 벡터 표현 / 여기서는 빈도 정보를 기반으로 하기에 CounterVectorizer 사용
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()
tf_train_features = tf_vectorizer.fit_transform(train_texts) 
tf_test_features = tf_vectorizer.transform(test_texts)

In [13]:
from sklearn.linear_model import LogisticRegression

In [19]:
lr_tf_l1 = LogisticRegression(C=0.5, penalty='l1', solver='saga', max_iter=10000) 

In [23]:
lr_tf_l1.fit(tf_train_features, train_labels) # 학습

LogisticRegression(C=0.5, max_iter=10000, penalty='l1', solver='saga')

In [25]:
pred_labels_tf_l1 = lr_tf_l1.predict(tf_test_features) # 예측

In [27]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(test_labels, pred_labels_tf_l1)

array([[ 871,  233],
       [ 126, 1061]], dtype=int64)

In [31]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(test_labels, pred_labels_tf_l1))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      1104
           1       0.82      0.89      0.86      1187

    accuracy                           0.84      2291
   macro avg       0.85      0.84      0.84      2291
weighted avg       0.85      0.84      0.84      2291



In [35]:
# 하나의 영화평 예측하기
print(lr_tf_l1.predict_proba(tf_test_features[0]))
print(lr_tf_l1.predict(tf_test_features[0]))

[[0.3010266 0.6989734]]
[1]


## 긍부정 단어 확인하기

In [38]:
# 단어 사전 만들기
words_dict = {}
for word, index in tf_vectorizer.vocabulary_.items():
    words_dict[index]=word

In [40]:
# Get coefficients of the model
coefficients = lr_tf_l1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
꿀잼 (3.447)
감탄 (2.836)
재밌게 (2.829)
재밌고 (2.562)
재미있게 (2.560)
존잼 (2.487)
재밌었 (2.404)
개꿀잼 (2.366)
여운 (2.349)
지루하지 (2.314)
재밌어 (2.230)
감사합 (2.202)
재미있었 (2.163)
빠르 (2.158)
흥미진진 (2.062)
강추 (2.055)
최고 (1.950)
사랑해 (1.947)
재미있어 (1.900)
재밋어 (1.864)
즐겁 (1.842)
유쾌 (1.841)
아름다운 (1.826)
재미있네 (1.818)
심장 (1.800)
낮아 (1.788)
감명 (1.755)
재미있고 (1.749)
재미있 (1.746)
재밌는 (1.704)

부정 단어 상위 30 개
갑자기 (-1.961)
지루해서 (-1.978)
댓글알바 (-1.991)
알바 (-2.028)
그닥 (-2.065)
지루하다 (-2.072)
별로 (-2.098)
왜곡 (-2.142)
졸았 (-2.143)
지루하고 (-2.147)
전형 (-2.161)
허무 (-2.184)
억지 (-2.200)
아까 (-2.234)
신파극 (-2.257)
짜증 (-2.308)
아까운 (-2.352)
재미없 (-2.366)
엉망 (-2.405)
망작 (-2.499)
지루했 (-2.501)
팔이 (-2.522)
역사왜곡 (-2.674)
거르 (-2.675)
쓰레기 (-2.737)
아까웠 (-2.802)
이하 (-3.005)
차라리 (-3.336)
노잼 (-3.620)
최악 (-4.687)


In [42]:
lr_tf_l2 = LogisticRegression(C=0.5, penalty='l2', solver='saga', max_iter=10000)

In [44]:
lr_tf_l2.fit(tf_train_features, train_labels) # 학습

LogisticRegression(C=0.5, max_iter=10000, solver='saga')

In [45]:
pred_labels_tf_l2 = lr_tf_l2.predict(tf_test_features)

In [46]:
print(classification_report(test_labels, pred_labels_tf_l2))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1104
           1       0.85      0.88      0.86      1187

    accuracy                           0.86      2291
   macro avg       0.86      0.86      0.86      2291
weighted avg       0.86      0.86      0.86      2291



In [47]:
# Get coefficients of the model
coefficients = lr_tf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌게 (2.619)
꿀잼 (2.497)
재미있게 (2.241)
재밌었 (2.196)
재밌어 (2.055)
여운 (1.967)
재미있었 (1.916)
최고 (1.906)
재밌고 (1.835)
강추 (1.782)
감사합 (1.722)
지루하지 (1.704)
감탄 (1.674)
재미있어 (1.655)
존잼 (1.585)
아름다운 (1.576)
재밌는 (1.564)
개꿀잼 (1.529)
재미있 (1.522)
가는 (1.510)
흥미진진 (1.509)
재밋어 (1.501)
재밌 (1.476)
가슴 (1.466)
지루할 (1.409)
재미있네 (1.403)
즐겁 (1.375)
대박 (1.370)
유쾌 (1.357)
심장 (1.330)

부정 단어 상위 30 개
비추 (-1.439)
지루하다 (-1.521)
지루해서 (-1.541)
아깝 (-1.583)
지루하고 (-1.617)
유치 (-1.618)
그닥 (-1.641)
엉망 (-1.645)
졸았 (-1.654)
자고 (-1.663)
실망 (-1.690)
허무 (-1.705)
아까웠 (-1.718)
거르 (-1.723)
아까 (-1.744)
팔이 (-1.770)
망작 (-1.785)
역사왜곡 (-1.817)
알바 (-1.833)
짜증 (-1.884)
아까운 (-1.909)
별로 (-2.026)
지루했 (-2.028)
억지 (-2.028)
이하 (-2.050)
재미없 (-2.271)
차라리 (-2.290)
쓰레기 (-2.395)
노잼 (-3.173)
최악 (-3.653)
